<a href="https://colab.research.google.com/github/amoyag/Biofisica/blob/main/S1_introduccion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#@title Setup mínimo para Colab (NumPy, SciPy, Matplotlib, Pandas)
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use('seaborn-v0_8')


## Ecuaciones diferenciales para la simulación de circuitos biológicos.
En biología de sistemas, las concentraciones de mRNA/proteína cambian en el tiempo según producción y degradación/dilución. Un gen regulado simple se modela como:

$$dYdt=P(t)−α Y\frac{dY}{dt} = P(t) - \alpha\$$

$$YdtdY​=P(t)−αY$$

donde $P(t)$ es la tasa efectiva de producción (que puede depender de entradas/reguladores) y $\alpha$ es la tasa de pérdida. Esta formulación permite estudiar tiempos de respuesta, estados estacionarios y robustez de los circuitos$^1$.

En el taller 3, discutimos que proteínas y enzimas pueden comportarse como elementos de computación; las ODEs capturan esa computación distribuida sin “cables”, mediada por difusiones y acoplamientos alostéricos.


[1. Ver capítulos 1 y 2 de Alon U. Introduction to systems Biology]

En estas prácticas vamos a usar una API mínima que puedas “montar” como LEGO:

```python
def model(t, y, params): — modelo ODE del circuito (devuelve la derivada de y respecto al tiempo).
```
```python
simulate(model, y0, t_span, t_eval, params) — integrador común (SciPy).
plot_timeseries(sol, labels) — gráfica unificada.
```
```python
Bloques de entrada (step_input, pulse_input) y funciones reguladoras (hill_activation, hill_repression)
```

Así podrás usar el código para realizar simulaciones de circuitos biológicos y entender sus propiedades con una compresión mínima de Python.

In [ ]:

#@title Bloques reutilizables: simulación y visualización

def simulate(model, y0, t_span, t_eval, params):
    """Integra un sistema ODE con solve_ivp."""
    sol = solve_ivp(lambda t, y: model(t, y, params),
                    t_span=t_span, y0=y0, t_eval=t_eval,
                    method='RK45', rtol=1e-6, atol=1e-9)
    return sol

def plot_timeseries(sol, labels=None, title="Dinámica temporal"):
    plt.figure(figsize=(7,4))
    for i in range(sol.y.shape[0]):
        lab = labels[i] if labels and i < len(labels) else f"var{i}"
        plt.plot(sol.t, sol.y[i], label=lab)
    plt.xlabel("Tiempo")
    plt.ylabel("Concentración (a.u.)")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Regulación tipo Hill (activación y represión)
def hill_activation(x, K, n):
    """f(x) = x^n / (K^n + x^n)"""
    return (x**n) / (K**n + x**n)

def hill_repression(x, K, n):
    """f_rep(x) = 1 / (1 + (x/K)^n)"""
    return 1.0 / (1.0 + (x / K)**n)

# Entradas (estímulos)
def step_input(t, t_on=10.0, val_before=0.0, val_after=1.0):
    return val_before if t < t_on else val_after

def pulse_input(t, t_on=10.0, t_off=20.0, val=1.0):
    return val if t_on <= t < t_off else 0.0
